In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from utils.visdom_utils import VisFunc

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
mb_size = 100

X_dim = 28 * 28
h_dim = 128
z_dim = 10

cnt = 0
lr = 1e-4

# MNIST Data

In [ ]:
transform = transforms.ToTensor()
mnist = datasets.MNIST(root='./dataset2',
                       train=True,
                       transform=transform,
                       download=True)
mnist_loader = DataLoader(mnist, batch_size=mb_size)

In [ ]:
len(mnist)

In [ ]:
len(mnist_loader)

# Define Model

In [ ]:
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim),
    torch.nn.Sigmoid()
)

# Critic 
D = torch.nn.Sequential(
    torch.nn.Linear(X_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, 1),
)

In [ ]:
def reset_grad():
    G.zero_grad()
    D.zero_grad()

In [ ]:
# Optimizer for Generator
G_solver = optim.RMSprop(G.parameters(), lr=lr)

# Optimizer for Critic
D_solver = optim.RMSprop(D.parameters(), lr=lr)

# Visdom
env_name = 'WGAN'
vf = VisFunc(enval=env_name)

In [ ]:
for epoch in range(200):
    mnist_iter = iter(mnist_loader)
    it = 0
    while it < len(mnist_loader):

        ################
        # Train Critic #
        ################
        for _ in range(5):
            # Sample data
            z = Variable(torch.randn(mb_size, z_dim))
            X, _ = next(mnist_iter)
            X = X.view(mb_size, -1)
            X = Variable(X)

            # forward-loss-backward-update
            G_sample = G(z)
            D_real = D(X)
            D_fake = D(G_sample)

            D_loss = -(torch.mean(D_real) - torch.mean(D_fake))

            D_loss.backward()
            D_solver.step()

            # Weight clipping
            for p in D.parameters():
                p.data.clamp_(-0.01, 0.01)

            # Reset gradient
            reset_grad()
            
            it += 1

        ###################
        # Train Generator #
        ###################

        # forward-loss-backward-update
        z = Variable(torch.randn(mb_size, z_dim))

        G_sample = G(z)
        D_fake = D(G_sample)

        G_loss = -torch.mean(D_fake)

        G_loss.backward()
        G_solver.step()

        # Reset gradient
        reset_grad()

        # Print and plot
        if it % 50 == 0:
            print('Epoch {} | [{}/{}] | D_loss: {:.2f} | G_loss: {:.2f}'
                  .format(epoch, it, len(mnist_loader),
                          float(D_loss.data), float(G_loss.data)))
            
            samples = G(z).data.numpy()[:100]

            fig = plt.figure(figsize=(10, 10))
            gs = gridspec.GridSpec(10, 10)
            gs.update(wspace=0.05, hspace=0.05)

            for i, sample in enumerate(samples):
                ax = plt.subplot(gs[i])
                plt.axis('off')
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                ax.set_aspect('equal')
                plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

            if not os.path.exists('out/'):
                os.makedirs('out/')

            plt.savefig('out/{}.png'.format(str(cnt).zfill(3)), bbox_inches='tight')
           # vf.imshow_multi(sample.data, nrow=10, title='', factor=1)            

            cnt += 1
            plt.close(fig)